In [1]:
import numpy as np

def generate_log2system(int_length, fraction_length):
  #system 2^(I.F)
  print ("generating table for 2^(I.F), I=%d, F=%d "%(int_length,fraction_length))
  step  =  2**(-fraction_length)
  power_table = np.arange(-2**int_length,  2**int_length, step)
  print ("power 2 table: ", list(power_table))
  table = list(map(lambda x: 2.0**x, power_table))
  print ("system 2^(%d.%d): "%(int_length,fraction_length), table)
  return table

weight_table = generate_log2system(2,1)


generating table for 2^(I.F), I=2, F=1 
power 2 table:  [-4.0, -3.5, -3.0, -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5]
system 2^(2.1):  [0.0625, 0.08838834764831845, 0.125, 0.1767766952966369, 0.25, 0.3535533905932738, 0.5, 0.7071067811865476, 1.0, 1.4142135623730951, 2.0, 2.8284271247461903, 4.0, 5.656854249492381, 8.0, 11.313708498984761]


In [2]:
import os
!nvidia-smi
!pip install transformers datasets
!pip install nlp

!git clone https://github.com/minhhn2910/QPyTorch
try:
  os.chdir('/content/QPyTorch')
except:
  pass
!git checkout posit-constant-generation

!pip install ./
!pip install ninja

try:
  os.chdir('/content/')
except:
  pass

import torch

if not torch.cuda.is_available():
  raise RuntimeError('Cannot run this cell without GPU runtime.')

import torch
import torch.nn as nn
import numpy as np




Wed May 21 17:36:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.02              Driver Version: 576.02         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060        On  |   00000000:03:00.0  On |                  N/A |
| 58%   53C    P3             28W /  170W |    2478MiB /  12288MiB |     17%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
device = 'cuda'
model_id = 'gpt2-large'
model = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)
# Print all layer names
print("Listing all layers in the model:")
for name, module in model.named_modules():
    print(name, ":", type(module))

/home/yimin/miniconda3/envs/posit/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-21 17:36:06.163432: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-21 17:36:06.170258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-21 17:36:06.178543: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-21 17:36:

Listing all layers in the model:
 : <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>
transformer : <class 'transformers.models.gpt2.modeling_gpt2.GPT2Model'>
transformer.wte : <class 'torch.nn.modules.sparse.Embedding'>
transformer.wpe : <class 'torch.nn.modules.sparse.Embedding'>
transformer.drop : <class 'torch.nn.modules.dropout.Dropout'>
transformer.h : <class 'torch.nn.modules.container.ModuleList'>
transformer.h.0 : <class 'transformers.models.gpt2.modeling_gpt2.GPT2Block'>
transformer.h.0.ln_1 : <class 'torch.nn.modules.normalization.LayerNorm'>
transformer.h.0.attn : <class 'transformers.models.gpt2.modeling_gpt2.GPT2SdpaAttention'>
transformer.h.0.attn.c_attn : <class 'transformers.pytorch_utils.Conv1D'>
transformer.h.0.attn.c_proj : <class 'transformers.pytorch_utils.Conv1D'>
transformer.h.0.attn.attn_dropout : <class 'torch.nn.modules.dropout.Dropout'>
transformer.h.0.attn.resid_dropout : <class 'torch.nn.modules.dropout.Dropout'>
transformer.h.0.ln_2 : <clas

In [ ]:
from transformers import pipeline
from transformers import set_seed
from qtorch_plus.quant import posit_quantize, float_quantize, configurable_table_quantize
import transformers
transformers.logging.set_verbosity_error()



#final with 3 layers skipped :
full_table = np.array([6.59179688e-03, 2.19726562e-02, 4.02832031e-02, 6.25000000e-02,
                8.51440430e-02, 2.10937500e-01, 1.21093750e-01, 5.36407471e-01,
                1.56250000e-02, 6.25000000e-02, 1.25000000e-01, 1.64062500e-01,
                2.92968750e-01, 6.25000000e-01, 8.43750000e-01, 4.37500000e-01,
                1.00000000e+00, 2.11486816e+00, 1.50000000e+00, 1.25000000e+00,
                4.25000000e+00, 3.08789062e+00, 8.25000000e+00, 6.00000000e+00])

weight_table = full_table[:8]
act_table = full_table[8:]
weight_table = np.sort(weight_table)
act_table = np.sort(act_table)
print (len(weight_table))
print (weight_table)
print (act_table)

def linear_weight(input):
    # return input
    # return configurable_table_quantize(input, torch.tensor(weight_table,dtype = torch.float), scale= 1.0)
    return posit_quantize(input,nsize=8, es=1, scale = 1)
    # return float_quantize(input,exp=4, man=3, rounding="nearest")

def linear_activation(input):
    # return input
    # return configurable_table_quantize(input,torch.tensor(act_table, dtype=torch.float), scale= 1.0)
    # return posit_quantize(input,nsize=6, es=0, scale = 1)
    return float_quantize(input,exp=4, man=3, rounding="nearest")

def forward_pre_hook_linear(m, input):
    return (linear_activation(input[0]),)

model = model.to(device)
layer_count = 0
op_count = 0
epsilon = 1e-12  # To avoid log(0)
import transformers.modeling_utils as modeling_utils

for name, module in model.named_modules():
    # Check if the module is quantizable
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear) or isinstance(module, modeling_utils.Conv1D):
        layer_count += 1
        print(f"Processing layer: {name}")

        # Flatten the weights and compute log2 scale
        weights_flattened = module.weight.data.cpu().numpy().flatten()
        # ////////////////////////////////////////////////////////////
        log2_weights = np.log2(np.abs(weights_flattened) + epsilon)
        counts, bins = np.histogram(log2_weights, bins=100)
        max_bin_index = np.argmax(counts)
        x_with_max_frequency = (bins[max_bin_index] + bins[max_bin_index + 1]) / 2  # Bin center
        print(f"x_with_max_frequency for {name}: {x_with_max_frequency:.2f}")
        scale = 2 ** (-x_with_max_frequency)
        # ////////////////////////////////////////////////////////////
        quantized_weights = posit_quantize(torch.tensor(module.weight.data, dtype=torch.float32), nsize=4, es=0, scale=scale)
        module.weight.data = quantized_weights

        print(f"Quantized layer {name} with scale factor: {scale:.2e}")
        module.register_forward_pre_hook(forward_pre_hook_linear)


        # Count operations
        if isinstance(module, modeling_utils.Conv1D):
            op_count += module.weight.shape[0] * module.weight.shape[1]
        else:
            op_count += module.in_features * module.out_features

    # Handle embeddings separately
    elif isinstance(module, nn.Embedding):
        print(f"Processing embedding layer: {name}")

        # Flatten the weights and compute log2 scale
        weights_flattened = module.weight.data.cpu().numpy().flatten()
        log2_weights = np.log2(np.abs(weights_flattened) + epsilon)

        # Compute histogram and find the bin with the maximum frequency
        counts, bins = np.histogram(log2_weights, bins=100)
        max_bin_index = np.argmax(counts)
        x_with_max_frequency = (bins[max_bin_index] + bins[max_bin_index + 1]) / 2  # Bin center
        print(f"x_with_max_frequency for {name}: {x_with_max_frequency:.2f}")

        # Apply quantization with scale based on x_with_max_frequency
        scale = 2 ** (-x_with_max_frequency)
        quantized_weights = posit_quantize(torch.tensor(module.weight.data, dtype=torch.float32), nsize=4, es=0, scale=scale)
        module.weight.data = quantized_weights

        print(f"Quantized embedding layer {name} with scale factor: {scale:.2e}")

print("Total layers processed:", layer_count)
print("MAC operation count:", op_count)

print("MAC operation count:", op_count)
print("Layer count:", layer_count)

print ("MAC operation count ", op_count)
print ("Layer count ", layer_count)


# from nlp import load_dataset

from datasets import load_dataset
test = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
encodings = tokenizer('\n\n'.join(test['text']), return_tensors='pt')
#model = model.to(device)
def generate_text(model_new):
  text_generation = pipeline("text-generation", model=model_new, tokenizer=tokenizer, device = 0)
  #set_seed(42)
  prefix_texts = ["Machine learning is the study of",
                    "In the 19th century, the invention",
                    "A robot was created",
                    "One day I will"
                  ]
  for prefix_text in prefix_texts:
    #generated_text= text_generation(prefix_text, max_length=50, do_sample=False )[0]
    set_seed(42)
    generated_text= text_generation(prefix_text, max_length=50, num_return_sequences=3)
    print(generated_text[0]['generated_text'])
    print ("-----------------")
    print(generated_text[1]['generated_text'])
    print ("-----------------------------------")
    print(generated_text[2]['generated_text'])
    print ("-----------------------------------")
generate_text(model)

print ("\n FP32 ref : \n")
# model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
# generate_text(model)



8
[0.0065918  0.02197266 0.0402832  0.0625     0.08514404 0.12109375
 0.2109375  0.53640747]
[0.015625   0.0625     0.125      0.1640625  0.29296875 0.4375
 0.625      0.84375    1.         1.25       1.5        2.11486816
 3.08789062 4.25       6.         8.25      ]
Processing embedding layer: transformer.wte
x_with_max_frequency for transformer.wte: -4.24
Quantized embedding layer transformer.wte with scale factor: 1.89e+01
Processing embedding layer: transformer.wpe
x_with_max_frequency for transformer.wpe: -11.83
Quantized embedding layer transformer.wpe with scale factor: 3.63e+03
Processing layer: transformer.h.0.attn.c_attn
x_with_max_frequency for transformer.h.0.attn.c_attn: -8.44
Quantized layer transformer.h.0.attn.c_attn with scale factor: 3.46e+02
Processing layer: transformer.h.0.attn.c_proj
x_with_max_frequency for transformer.h.0.attn.c_proj: -9.20
Quantized layer transformer.h.0.attn.c_proj with scale factor: 5.88e+02
Processing layer: transformer.h.0.mlp.c_fc
x_with_

/tmp/ipykernel_2800/2746343755.py:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  quantized_weights = posit_quantize(torch.tensor(module.weight.data, dtype=torch.float32), nsize=6, es=0, scale=scale)
/tmp/ipykernel_2800/2746343755.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  quantized_weights = posit_quantize(torch.tensor(module.weight.data, dtype=torch.float32), nsize=6, es=0, scale=scale)


x_with_max_frequency for transformer.h.0.mlp.c_proj: -9.02
Quantized layer transformer.h.0.mlp.c_proj with scale factor: 5.20e+02
Processing layer: transformer.h.1.attn.c_attn
x_with_max_frequency for transformer.h.1.attn.c_attn: -8.56
Quantized layer transformer.h.1.attn.c_attn with scale factor: 3.78e+02
Processing layer: transformer.h.1.attn.c_proj
x_with_max_frequency for transformer.h.1.attn.c_proj: -8.97
Quantized layer transformer.h.1.attn.c_proj with scale factor: 5.00e+02
Processing layer: transformer.h.1.mlp.c_fc
x_with_max_frequency for transformer.h.1.mlp.c_fc: -8.40
Quantized layer transformer.h.1.mlp.c_fc with scale factor: 3.38e+02
Processing layer: transformer.h.1.mlp.c_proj
x_with_max_frequency for transformer.h.1.mlp.c_proj: -8.65
Quantized layer transformer.h.1.mlp.c_proj with scale factor: 4.02e+02
Processing layer: transformer.h.2.attn.c_attn
x_with_max_frequency for transformer.h.2.attn.c_attn: -8.78
Quantized layer transformer.h.2.attn.c_attn with scale factor: 4

In [5]:
# torch.cuda.empty_cache()

import torch
from tqdm import tqdm

max_length = model.config.n_positions
stride = 1024

lls = []
input_size = encodings.input_ids.size(1)
print("Input size:", input_size)

for i in tqdm(range(0, input_size, stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, input_size)
    trg_len = end_loc - i

    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        log_likelihood = outputs[0] * trg_len
        # print(f"Log Likelihood for step {i}: {log_likelihood}")

    lls.append(log_likelihood)

if lls:  # Ensure lls is not empty
    ppl = torch.exp(torch.stack(lls).sum() / input_size)
    print("Perplexity:", ppl.item())
else:
    print("No log likelihoods calculated.")


Input size: 287644


100%|██████████| 281/281 [01:34<00:00,  2.99it/s]


Perplexity: 23.910572052001953
